Library Imports

In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow import keras
import seaborn as sns
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Activation,AveragePooling2D,BatchNormalization
import time
import numpy as np
import os
import pandas as pd
from keras.preprocessing.image import image_dataset_from_directory
from tensorflow.keras.layers.experimental.preprocessing import Rescaling
from sklearn.metrics import precision_score, accuracy_score, recall_score, confusion_matrix, ConfusionMatrixDisplay

2022-07-08 20:11:25.462251: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-07-08 20:11:25.462285: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


# Tensorflow 

In [3]:
def get_file_size(file_path):
    size = os.path.getsize(file_path)
    return size

In [4]:
def convert_bytes(size, unit=None):
    if unit == "KB":
        return print('File size: ' + str(round(size / 1024, 3)) + ' Kilobytes')
    elif unit == "MB":
        return print('File size: ' + str(round(size / (1024 * 1024), 3)) + ' Megabytes')
    else:
        return print('File size: ' + str(size) + 'bytes')
        

In [5]:
# Loading the dataset from directory
train_path = 'maize_dataset/train'
train_dataset = image_dataset_from_directory(directory="maize_dataset/train",image_size=(256,256)) 
valid_dataset = image_dataset_from_directory(directory="maize_dataset/valid",image_size=(256,256))
test_dataset = image_dataset_from_directory(directory="maize_dataset/test", image_size=(256,256))

classes = os.listdir(train_path)
print('classes: ', classes)

Found 7316 files belonging to 4 classes.
Found 1829 files belonging to 4 classes.
Found 3 files belonging to 1 classes.
classes:  ['Corn_(maize)___Northern_Leaf_Blight', 'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot', 'Corn_(maize)___healthy', 'Corn_(maize)___Common_rust_']


2022-07-08 20:11:44.288469: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-07-08 20:11:44.288503: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-07-08 20:11:44.288527: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (kali): /proc/driver/nvidia/version does not exist
2022-07-08 20:11:44.288813: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
# Rescalling
rescale = Rescaling(scale=1.0/255)
train_dataset = train_dataset.map(lambda image,label:(rescale(image),label))
valid_dataset  = valid_dataset.map(lambda image,label:(rescale(image),label))

In [7]:
# Modelling
model = keras.Sequential()
model.add(keras.layers.Conv2D(32,(3,3),activation="relu",padding="same",input_shape=(256,256,3)))
model.add(keras.layers.Conv2D(32,(3,3),activation="relu",padding="same"))
model.add(keras.layers.MaxPooling2D(3,3))

model.add(keras.layers.Conv2D(64,(3,3),activation="relu",padding="same"))
model.add(keras.layers.Conv2D(64,(3,3),activation="relu",padding="same"))
model.add(keras.layers.MaxPooling2D(3,3))

model.add(keras.layers.Conv2D(128,(3,3),activation="relu",padding="same"))
model.add(keras.layers.Conv2D(128,(3,3),activation="relu",padding="same"))
model.add(keras.layers.MaxPooling2D(3,3))

model.add(keras.layers.Conv2D(256,(3,3),activation="relu",padding="same"))
model.add(keras.layers.Conv2D(256,(3,3),activation="relu",padding="same"))

model.add(keras.layers.Conv2D(512,(5,5),activation="relu",padding="same"))
model.add(keras.layers.Conv2D(512,(5,5),activation="relu",padding="same"))

model.add(keras.layers.Flatten())

model.add(keras.layers.Dense(1568,activation="relu"))
model.add(keras.layers.Dropout(0.5))

model.add(keras.layers.Dense(38,activation="softmax"))

2022-07-08 20:11:44.874287: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 260112384 exceeds 10% of free system memory.
2022-07-08 20:11:45.073767: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 260112384 exceeds 10% of free system memory.
2022-07-08 20:11:45.147112: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 260112384 exceeds 10% of free system memory.


In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 256, 256, 32)      896       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 256, 256, 32)      9248      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 85, 85, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 85, 85, 64)        18496     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 85, 85, 64)        36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 28, 28, 64)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 28, 28, 128)       7

In [11]:
#model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
# Compile and train the model
model.compile(optimizer='adam',loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])


In [12]:
#model.fit(train_dataset, validation_data=valid_dataset, epochs = 5)
model.fit(train_dataset,validation_data=valid_dataset,epochs=5)

Epoch 1/5


/home/kali/.local/lib/python3.9/site-packages/keras/backend.py:4906: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  warnings.warn(
2022-07-08 20:19:39.130498: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 260112384 exceeds 10% of free system memory.
2022-07-08 20:19:39.212506: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 260112384 exceeds 10% of free system memory.


229/229 [==============================] - 1824s 8s/step - loss: 1.9862 - accuracy: 0.3420 - val_loss: 0.8672 - val_accuracy: 0.5637
Epoch 2/5
229/229 [==============================] - 1821s 8s/step - loss: 0.7892 - accuracy: 0.5678 - val_loss: 0.8029 - val_accuracy: 0.5746
Epoch 3/5
229/229 [==============================] - 1796s 8s/step - loss: 0.7690 - accuracy: 0.5715 - val_loss: 0.7913 - val_accuracy: 0.5703
Epoch 4/5
229/229 [==============================] - 1797s 8s/step - loss: 0.7740 - accuracy: 0.5790 - val_loss: 0.8313 - val_accuracy: 0.5686
Epoch 5/5
229/229 [==============================] - 1790s 8s/step - loss: 0.8095 - accuracy: 0.5616 - val_loss: 0.9966 - val_accuracy: 0.5659


In [ ]:
# How the network looks

In [ ]:
# Training


In [ ]:
# Visualization
plt.plot(model.history["accuracy"])
plt.plot(model.history["val_accuracy"])
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.title("Maize Model Accuracy")
plt.legend(["Train", "Validation"])
plt.show()

In [ ]:
plt.plot(model.history["loss"])
plt.plot(model.history["val_loss"])
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.title("Maize Model Loss")
plt.legend(["Train", "Validation"])
plt.show()

In [ ]:
# TF LITE Model
MODEL_NAME = "maize.h5"
model.save(MODEL_NAME)
convert_bytes(get_file_size(MODEL_NAME),"MB")

In [ ]:
import tensorflow as tf
MAIZE_TFLITE = "maize.tflite"
tf_lite_converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = tf_lite_converter.convert()

In [ ]:
tflite_model_name = MAIZE_TFLITE
open(tflite_model_name, "wb").write(tflite_model)
convert_bytes(get_file_size(MAIZE_TFLITE), "KB")